In [10]:
import pandas as pd
import numpy as np

# stats to calculate
- ## outsourcing:
  - percentage of books scanned at BPOs per year - done!
- ## pay
    - calculations on maximum wages workers at cebu could have made based on number of workers there per month and the contract IA had with Innodata at the time (at 1.5million per year / 12 months with x workers working per month, that leaves what max monthly wage).
- ## rates of work:
  - as measure of pages scanned to workers working per center per month
  - avergage pages scanned to workers working per center + standard deviation
- ## turn over rates- how long do operators stay at their jobs across centers
  - avg. days worked by workers at each center + standard deviation - will need to throw out lots of junk

In [3]:
# accessing the geocoded texts dataset from my external hard drive. You can get it from this box link: 
texts = pd.read_csv('/Volumes/Samsung_T5/scanning_labor_in_IA/geocoded-texts-data.csv', low_memory=False)
texts = texts.drop(columns=['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'])

# Outsourcing

In [7]:
# scans per center per month
scans_center_month = pd.read_csv("https://raw.githubusercontent.com/ers6/ia_scanning_labor_data/main/metadata-analysis/metadata-records-analysis-csvs/scans_per_center_per_month.csv") 

# pages per center per month
pages_center_month = pd.read_csv("https://raw.githubusercontent.com/ers6/ia_scanning_labor_data/main/metadata-analysis/metadata-records-analysis-csvs/pages_per_center_per_month.csv")

# scans per center type per month
scans_type_month = pd.read_csv("https://raw.githubusercontent.com/ers6/ia_scanning_labor_data/main/metadata-analysis/metadata-records-analysis-csvs/scans_per_center_type_per_month.csv").dropna(subset=['center_type'])
scans_type_month = scans_type_month[['month_year','center_type', 'books_scanned']]


## Annual Counts + Percentage of Scans by Center Type

In [8]:
scans_type_year = scans_type_month.groupby(['center_type','month_year']).sum().reset_index()

In [9]:
scans_type_year['year'] = ""

for i in range(len(scans_type_year)):
    scans_type_year.at[i, 'year'] = str(scans_type_year.at[i,'month_year']).split('-')[0]

In [10]:
scans_type_year = scans_type_year.groupby(['center_type','year']).sum().reset_index()[['center_type', 'year', 'books_scanned']]

In [11]:
scans_type_year

,center_type,year,books_scanned
0,academic,2001,3
1,academic,2002,421
2,academic,2006,37248
3,academic,2007,108590
4,academic,2008,164714
...,...,...,...
118,public,2019,21036
119,public,2020,8342
120,public,2021,16017
121,public,2022,29775


In [14]:
scans_per_year = scans_type_year.groupby(['year']).sum().reset_index()[['year','books_scanned']]
scans_per_year.loc[scans_per_year['year']== '2010']['books_scanned'].tolist()[0]

261573

In [8]:
scans_type_year['percent'] = ''

for i in range(len(scans_type_year)):
    scans_type_year.at[i, 'percent'] = scans_type_year.at[i, 'books_scanned'] / scans_per_year.loc[scans_per_year['year'] == scans_type_year.at[i, 'year']]['books_scanned'].tolist()[0]

 

In [9]:
scans_type_year.to_csv('/Users/elizabethschwartz/Documents/GitHub/ia_scanning_labor_data/metadata-analysis/metadata-records-analysis-csvs/percentage_of_books_scanned_per_center_type_per_year.csv')

In [10]:
scans_type_year

,center_type,year,books_scanned,percent
0,academic,2001,3,1.0
1,academic,2002,421,0.97907
2,academic,2006,37248,0.589554
3,academic,2007,108590,0.683085
4,academic,2008,164714,0.639723
...,...,...,...,...
118,public,2019,21036,0.043464
119,public,2020,8342,0.010275
120,public,2021,16017,0.016478
121,public,2022,29775,0.027982


In [13]:
scans_type_year['center_type'].tolist()
this_dict = {'year':"",'academic':"", 'bpo':"", 'government':"", 'hq':"", 'museum':"", 'public':"", 'total_books':""}
year_list = []
for i in range(20): 
    year_list.append(this_dict)

year_list

[{'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'museum': '',
  'public': '',
  'total_books': ''},
 {'year': '',
  'academic': '',
  'bpo': '',
  'government': '',
  'hq': '',
  'mus

In [12]:
def get_center_count(center_type, year): 
    try:
        result = scans_type_year.loc[(scans_type_year['center_type'] == center_type) & (scans_type_year['year'] == year)].reset_index().to_dict()['books_scanned'][0]
    except KeyError: 
        result = 0
    return result


def get_center_percent(center_type, year):
    try:
        result = scans_type_year.loc[(scans_type_year['center_type'] == center_type) & (scans_type_year['year'] == year)].reset_index().to_dict()['percent'][0]
        result = f"{result:.2%}"
    except KeyError: 
        result = 0
    return result


In [13]:
get_center_count('academic', '1998')

0

In [27]:
the_years = scans_per_year['year'].tolist()
yearly_counts = []
for this_year in the_years: 
    yearly_counts.append({'year': this_year,
                         'academic': get_center_count('academic', this_year),
                         'bpo': get_center_count('bpo', this_year), 
                         'government': get_center_count('government', this_year), 
                         'hq': get_center_count('hq', this_year), 
                         'museum': get_center_count('museum', this_year),
                         'public': get_center_count('public', this_year), 
                         'total_books':""})

yearly_percents = []
for this_year in the_years: 
    yearly_percents.append({'year': this_year,
                         'academic': get_center_percent('academic', this_year),
                         'bpo': get_center_percent('bpo', this_year), 
                         'government': get_center_percent('government', this_year), 
                         'hq': get_center_percent('hq', this_year), 
                         'museum': get_center_percent('museum', this_year),
                         'public': get_center_percent('public', this_year)})
  

In [152]:
the_years = scans_per_year['year'].tolist()

In [16]:
scans_type_year.loc[(scans_type_year['center_type'] == 'academic') & (scans_type_year['year'] == '2012')]

,center_type,year,books_scanned,percent
8,academic,2012,91679,0.250427


In [28]:
yearly_percents

[{'year': '2001',
  'academic': '100.00%',
  'bpo': 0,
  'government': 0,
  'hq': 0,
  'museum': 0,
  'public': 0},
 {'year': '2002',
  'academic': '97.91%',
  'bpo': 0,
  'government': '0.93%',
  'hq': 0,
  'museum': '0.23%',
  'public': '0.93%'},
 {'year': '2006',
  'academic': '58.96%',
  'bpo': 0,
  'government': 0,
  'hq': '41.04%',
  'museum': '0.00%',
  'public': 0},
 {'year': '2007',
  'academic': '68.31%',
  'bpo': 0,
  'government': '0.25%',
  'hq': '28.38%',
  'museum': '0.57%',
  'public': '2.49%'},
 {'year': '2008',
  'academic': '63.97%',
  'bpo': 0,
  'government': '9.77%',
  'hq': '12.01%',
  'museum': '0.59%',
  'public': '13.66%'},
 {'year': '2009',
  'academic': '56.18%',
  'bpo': '0.00%',
  'government': '17.74%',
  'hq': '9.40%',
  'museum': '0.59%',
  'public': '15.96%'},
 {'year': '2010',
  'academic': '51.05%',
  'bpo': '2.18%',
  'government': '9.80%',
  'hq': '22.61%',
  'museum': '0.30%',
  'public': '13.93%'},
 {'year': '2011',
  'academic': '31.95%',
  'bpo

In [19]:
yearly_counts_df = pd.DataFrame.from_records(yearly_counts) 
yearly_counts_df


,year,academic,bpo,government,hq,museum,public,total_books
0,2001,3,0,0,0,0,0,
1,2002,421,0,4,0,1,4,
2,2006,37248,0,0,25931,1,0,
3,2007,108590,0,402,45116,899,3963,
4,2008,164714,0,25163,30925,1510,35165,
5,2009,135579,11,42811,22681,1414,38515,
6,2010,133532,5714,25634,59152,784,36441,
7,2011,98462,130463,30265,25,465,44831,
8,2012,91679,188778,35093,685,282,39508,
9,2013,89153,20919,48684,64,1819,36198,


In [31]:
yearly_percents_df = pd.DataFrame.from_records(yearly_percents) 

In [32]:
yearly_percents_df

,year,academic,bpo,government,hq,museum,public
0,2001,100.00%,0,0,0,0,0
1,2002,97.91%,0,0.93%,0,0.23%,0.93%
2,2006,58.96%,0,0,41.04%,0.00%,0
3,2007,68.31%,0,0.25%,28.38%,0.57%,2.49%
4,2008,63.97%,0,9.77%,12.01%,0.59%,13.66%
5,2009,56.18%,0.00%,17.74%,9.40%,0.59%,15.96%
6,2010,51.05%,2.18%,9.80%,22.61%,0.30%,13.93%
7,2011,31.95%,42.34%,9.82%,0.01%,0.15%,14.55%
8,2012,25.04%,51.57%,9.59%,0.19%,0.08%,10.79%
9,2013,45.14%,10.59%,24.65%,0.03%,0.92%,18.33%


In [33]:
yearly_counts_df.to_csv('/Users/elizabethschwartz/Documents/GitHub/ia_scanning_labor_data/metadata-analysis/metadata-records-analysis-csvs/count_books_scanned_per_year_per_type.csv')
yearly_percents_df.to_csv('/Users/elizabethschwartz/Documents/GitHub/ia_scanning_labor_data/metadata-analysis/metadata-records-analysis-csvs/percentage_books_scanned_per_year_per_type.csv')


## Overall Count + Percent of Books Scanned by Center Type

In [47]:
scans_type_year

total_scans_by_type =  scans_type_year.groupby(['center_type']).sum().reset_index()[['center_type','books_scanned']]

In [48]:
total_scans_by_type

,center_type,books_scanned
0,academic,1314866
1,archive,21134
2,bpo,4048959
3,government,700435
4,hq,185807
5,museum,35502
6,public,472975


In [51]:
total_scans_by_type['percent']= total_scans_by_type['books_scanned']/total_scans_by_type['books_scanned'].sum() * 100

In [52]:
total_scans_by_type

,center_type,books_scanned,percent
0,academic,1314866,19.394225
1,archive,21134,0.311726
2,bpo,4048959,59.721996
3,government,700435,10.331390
4,hq,185807,2.740646
5,museum,35502,0.523653
6,public,472975,6.976364


# Rate of Work

## Rate of Work as a Measure of Pages Scanned to Workers Working per Center per Month

### Books Scanned per Worker per Day 
This isn't a good way of understanding the amount of work being done per worker per day because some books are much shorter than others. David Sutton of Princeton's scanning center consistently scans some of the most books per day using this method because he scanned much of their postcard collection: https://archive.org/details/ar050jtanis 

However, the data is more complete than other measures. Only 4537 records are missing date informaiton or operator information.

In [4]:
texts['scan_day'] = pd.to_datetime(texts['scandate']).dt.to_period('D').astype('str')
texts['operator'] = texts['operator'].astype('str')
texts['operator'] = texts['operator'].map(str.lower)
scans_per_day = texts[['name', 'scan_day', 'identifier', 'operator']]

In [14]:
scans_per_day

,name,scan_day,identifier,operator
0,Datum Data Co. Ltd.,2013-10-10,geometrysuccessi00thom,scanner-shenzhen-leo@archive.org
1,Datum Data Co. Ltd.,2012-09-08,berdiemobilittde00hebe,scanner-shenzhen-mary@archive.org
2,Datum Data Co. Ltd.,2011-07-16,beilsteinshandb28beil,scanner-shenzhen-lina@archive.org
3,Datum Data Co. Ltd.,2012-08-14,theoryapplicatio00adva,scanner-shenzhen-dragon@archive.org
4,Datum Data Co. Ltd.,2012-09-21,specialeditionus00edbo_0,scanner-shenzhen-yan@archive.org
...,...,...,...,...
9029407,Perpustakaan Provinsi Bali,2012-02-23,pratekaning-wong-mara-mandeg,nan
9029408,Perpustakaan Provinsi Bali,2012-02-23,kramaning-caru-bang-bonggalan,nan
9029409,Perpustakaan Provinsi Bali,2012-03-12,puja-parikrama-panigang-sasih,nan
9029410,Perpustakaan Provinsi Bali,2012-03-12,puja-pancasanak-buda,nan


In [15]:
scans_per_worker_day = scans_per_day.groupby(['name','operator', 'scan_day']).count().reset_index()

In [16]:
missing_scans = scans_per_worker_day.loc[(scans_per_worker_day['scan_day'] == 'NaT')|(scans_per_worker_day['operator'] == 'nan')]

missing_scans

,name,operator,scan_day,identifier
0,1 Dollar Scan,nan,NaT,20
1729,Allen County Public Library Geneaology Center,associate-christine-calhoun@archive.org,NaT,1
3099,Allen County Public Library Geneaology Center,associate-janet-breitenwischer@archive.org,NaT,1
6516,Allen County Public Library Geneaology Center,associate-ladonna-hartmann@archive.org,NaT,18
8959,Allen County Public Library Geneaology Center,associate-rosa-guzman@archive.org,NaT,1
...,...,...,...,...
429151,Yiddish Book Center,scanner-mitt-raj@archive.org,NaT,10
429271,Yiddish Book Center,scanner-zoe-schacht-levine@archive.org,NaT,2
429291,Yiddish Book Center,volunteer-allison-posner@archive.org,NaT,6
429311,Yiddish Book Center,volunteer-jessica-parker@archive.org,NaT,5


In [6]:
# weeding out missing data- we want to throw out rows where operator = nan and/or scanday = nat
scans_per_worker_day = scans_per_worker_day.loc[scans_per_worker_day['scan_day'] != 'NaT']
scans_per_worker_day = scans_per_worker_day.loc[scans_per_worker_day['operator'] != 'nan']

In [7]:
scans_per_worker_day = scans_per_worker_day.rename(columns={'name':'name', 'operator': 'operator','scan_day':'scan_day','identifier':'books_scanned'})

In [8]:
scans_per_worker_day.sort_values(by="books_scanned", ascending=False)

,name,operator,scan_day,books_scanned
41669,Boston Public Library,book-creator-script@archive.org,2010-09-29,1975
41674,Boston Public Library,boston_uploads,2014-10-31,1162
314677,Princeton University,associate-david-sutton@archive.org,2014-06-24,573
372340,University of Alberta,microfilm-processing@archive.org,2012-11-30,458
372379,University of Alberta,microfilm-processing@archive.org,2013-02-05,437
...,...,...,...,...
273235,Internt Archive Physical Archive,scanner-ashley-anderson@archive.org,2009-09-21,1
273241,Internt Archive Physical Archive,scanner-ashley-anderson@archive.org,2009-09-28,1
273243,Internt Archive Physical Archive,scanner-ashley-anderson@archive.org,2009-10-02,1
273244,Internt Archive Physical Archive,scanner-august-bourre@archive.org,2006-09-28,1


## Pages Scanned per Worker per Day 
The imagecount measures how many images make up a scanned item. It corresponds to the actual number of images a worker captured as they pressed the capture button on the scribe machine, lowered the book using a foot pedal, flipped a page, and repeated. As such, it's a much better way to approximate rate of work. Unfortunately, the metadata field is often left blank in an IA metadata record. 2,284,561 records do not have any imagecount information and we exclude them from these analyses. 

In [9]:
pages = texts[['name', 'scan_day', 'imagecount', 'operator']]


In [6]:
missing_pages = pages.loc[(pages['scan_day'] == 'NaT') | (pages['operator'] == 'nan') |  (pages['imagecount'] == 'NaN')]

len(missing_pages)

2284561

In [13]:
pages = pages.loc[(pages['scan_day'] != 'NaT') & (pages['operator'] != 'nan') &  (pages['imagecount'] != 'NaN')]

In [14]:
pages

,name,scan_day,imagecount,operator
0,Datum Data Co. Ltd.,2013-10-10,200.0,scanner-shenzhen-leo@archive.org
1,Datum Data Co. Ltd.,2012-09-08,250.0,scanner-shenzhen-mary@archive.org
2,Datum Data Co. Ltd.,2011-07-16,962.0,scanner-shenzhen-lina@archive.org
3,Datum Data Co. Ltd.,2012-08-14,590.0,scanner-shenzhen-dragon@archive.org
4,Datum Data Co. Ltd.,2012-09-21,888.0,scanner-shenzhen-yan@archive.org
...,...,...,...,...
9028846,New York Botanical Garden,2017-12-08,450.0,operator1.nybg@archive.org
9028847,New York Botanical Garden,2017-04-05,4.0,operator1.nybg@archive.org
9028848,New York Botanical Garden,2017-12-20,530.0,operator1.nybg@archive.org
9028849,New York Botanical Garden,2017-05-11,86.0,operator1.nybg@archive.org


In [17]:
pages_per_worker_per_day = pages[['name', 'scan_day', 'imagecount', 'operator']]
pages_per_worker_per_day  = pages_per_worker_per_day .groupby(['name', 'scan_day', 'operator']).sum()
pages_per_worker_per_day  = pages_per_worker_per_day .reset_index()

pages_per_worker_per_day .sort_values(by="imagecount", ascending=False)


,name,scan_day,operator,imagecount
91690,Datum Data Co. Ltd.,2013-10-17,scanner-shenzhen-leo@archive.org,101942.0
82467,Datum Data Co. Ltd.,2011-07-08,scanner-shenzhen-leo@archive.org,92954.0
365119,University of Alberta,2010-01-25,zack@archive.org,86600.0
82421,Datum Data Co. Ltd.,2011-07-01,scanner-shenzhen-leo@archive.org,86450.0
91693,Datum Data Co. Ltd.,2013-10-22,scanner-shenzhen-leo@archive.org,84322.0
...,...,...,...,...
91370,Datum Data Co. Ltd.,2013-01-05,scanner-shenzhen-leo@archive.org,0.0
91347,Datum Data Co. Ltd.,2012-12-31,scanner-shenzhen-david@archive.org,0.0
91332,Datum Data Co. Ltd.,2012-12-28,scanner-shenzhen-thomas@archive.org,0.0
91302,Datum Data Co. Ltd.,2012-12-24,scanner-shenzhen-thomas@archive.org,0.0


In [18]:
# center level data 
pages_per_day = pages[['name', 'scan_day', 'imagecount']]
pages_per_day = pages_per_day.groupby(['name', 'scan_day']).sum()
pages_per_day = pages_per_day.reset_index()

pages_per_day

,name,scan_day,imagecount
0,Allen County Public Library Geneaology Center,2002-01-09,752.0
1,Allen County Public Library Geneaology Center,2002-01-10,108.0
2,Allen County Public Library Geneaology Center,2002-01-15,506.0
3,Allen County Public Library Geneaology Center,2008-05-15,564.0
4,Allen County Public Library Geneaology Center,2008-05-16,454.0
...,...,...,...
73314,Yiddish Book Center,2013-07-19,114.0
73315,Yiddish Book Center,2013-07-26,1663.0
73316,Yiddish Book Center,2013-07-29,180.0
73317,Yiddish Book Center,2013-08-02,1009.0


In [21]:
workers = pages[['name', 'scan_day', 'operator']]
workers = workers.drop_duplicates()
workers = workers.groupby(['name', 'scan_day']).count().reset_index()
workers

,name,scan_day,operator
0,Allen County Public Library Geneaology Center,2002-01-09,2
1,Allen County Public Library Geneaology Center,2002-01-10,1
2,Allen County Public Library Geneaology Center,2002-01-15,1
3,Allen County Public Library Geneaology Center,2008-05-15,1
4,Allen County Public Library Geneaology Center,2008-05-16,2
...,...,...,...
73314,Yiddish Book Center,2013-07-19,1
73315,Yiddish Book Center,2013-07-26,1
73316,Yiddish Book Center,2013-07-29,1
73317,Yiddish Book Center,2013-08-02,1


In [25]:

pages_stats = pd.merge(pages_per_day, workers, how='inner', on=['name', 'scan_day'])


In [26]:
pages_stats

,name,scan_day,imagecount,operator
0,Allen County Public Library Geneaology Center,2002-01-09,752.0,2
1,Allen County Public Library Geneaology Center,2002-01-10,108.0,1
2,Allen County Public Library Geneaology Center,2002-01-15,506.0,1
3,Allen County Public Library Geneaology Center,2008-05-15,564.0,1
4,Allen County Public Library Geneaology Center,2008-05-16,454.0,2
...,...,...,...,...
73314,Yiddish Book Center,2013-07-19,114.0,1
73315,Yiddish Book Center,2013-07-26,1663.0,1
73316,Yiddish Book Center,2013-07-29,180.0,1
73317,Yiddish Book Center,2013-08-02,1009.0,1


In [27]:
pages_stats['pages_to_worker_ratio'] = ''
i = 0
for i in range(len(pages_stats)): 
    pages_stats.at[i,'pages_to_worker_ratio'] = float(pages_stats.at[i, 'imagecount']) / float(pages_stats.at[i, 'operator']) 
    i += 0

In [28]:
pages_stats

,name,scan_day,imagecount,operator,pages_to_worker_ratio
0,Allen County Public Library Geneaology Center,2002-01-09,752.0,2,376.0
1,Allen County Public Library Geneaology Center,2002-01-10,108.0,1,108.0
2,Allen County Public Library Geneaology Center,2002-01-15,506.0,1,506.0
3,Allen County Public Library Geneaology Center,2008-05-15,564.0,1,564.0
4,Allen County Public Library Geneaology Center,2008-05-16,454.0,2,227.0
...,...,...,...,...,...
73314,Yiddish Book Center,2013-07-19,114.0,1,114.0
73315,Yiddish Book Center,2013-07-26,1663.0,1,1663.0
73316,Yiddish Book Center,2013-07-29,180.0,1,180.0
73317,Yiddish Book Center,2013-08-02,1009.0,1,1009.0


In [29]:
pages_stats.sort_values(by='pages_to_worker_ratio', ascending=False)
pages_stats = pages_stats.reset_index()

In [32]:
pages_stats.sort_values(by='pages_to_worker_ratio', ascending=False)

,index,name,scan_day,imagecount,operator,pages_to_worker_ratio
21194,21194,Datum Data Co. Ltd.,2013-10-17,101942.0,1,101942.0
63087,63087,University of Alberta,2010-01-25,86600.0,1,86600.0
21197,21197,Datum Data Co. Ltd.,2013-10-22,84322.0,1,84322.0
63756,63756,University of Alberta,2013-02-05,82095.0,1,82095.0
21199,21199,Datum Data Co. Ltd.,2013-10-24,78542.0,1,78542.0
...,...,...,...,...,...,...
28131,28131,Getty Research Institute Valencia Warehouse,2023-05-25,0.0,1,0.0
28129,28129,Getty Research Institute Valencia Warehouse,2023-05-23,0.0,1,0.0
31512,31512,Internet Archive Sheridan Headquarters,2008-04-18,0.0,1,0.0
28127,28127,Getty Research Institute Valencia Warehouse,2023-05-20,0.0,1,0.0


In [33]:
pages_stats.to_csv('/Users/e.schwartz/Documents/GitHub/ia_scanning_labor_data/pages_scanned_per_worker_per_day.csv')
workers.to_csv('/Users/e.schwartz/Documents/GitHub/ia_scanning_labor_data/number_workers_per_day.csv')

OSError: Cannot save file into a non-existent directory: '/Users/e.schwartz/Documents/GitHub/ia_scanning_labor_data'